In [1]:
%load_ext autoreload
%autoreload 2

# Q Learning in Tic Tac Toe

This code file is pretty much a code-with-answer-key session, where I try to recreate something others have done while having their own code to heavily reference. The point of this is to explore Q Learning. The code I structured my game after is [here](https://www.kaggle.com/slobo777/tic-tac-toe-agent-using-q-learning). 

So let's make a tic tac toe game first. A move is defined to be simply an override of the original board. The object does not provide good interface for human player, but is suited for the `Agent` to work with. 

In [8]:
class Game:
    empty = ' '
    first_move = 'X'
    second_move = 'O'

    def __init__(self, board_size=3):
        # board represented as a size*size length string, each char
        # being one of ' ', 'X', or 'O'
        self.board = " " * (board_size**2) 
        self.player = Game.first_move
        self.winner = None 
    
    @property
    def board_len(self):
        """ Length of one side of the board
        """
        return int(len(self.board)**0.5)

    def possible_boards(self):
        """ Gives a list of all possible configurations of the board
            after the current player moves
        """
        all_boards = []
        for i in range(len(self.board)):
            if self.board[i] == Game.empty:
                all_boards.append(self.board[:i] + self.player + self.board[i+1:])
        return all_boards
    
    def move(self, board):
        """ Given a new state of the board, makes a move. Enforces 
            tic tac toe rules
        """
        if self.winner is not None:
            raise Exception("The game is already completed. Cannot make another move")
        if board not in self.possible_boards():
            raise Exception("Cannot make move {} to {} for player {}".format(self.board, board, self.player))
        
        self.board = board
        self.winner = self.get_winner(board)
        if self.winner:
            self.player = None
        elif self.player == Game.first_move:
            self.player = Game.second_move
        elif self.player == Game.second_move:
            self.player = Game.first_move

    def get_winner(self, board):
        """ Takes a board and decides if there is a winner and which 
            player it is
        """
        lines_to_be_checked = []
        for i in range(self.board_len):
            lines_to_be_checked.append(list(range(i*self.board_len, (i+1)*self.board_len)))
            lines_to_be_checked.append(list(range(i, len(board), self.board_len)))
        lines_to_be_checked.append(list(range(0,len(board),self.board_len+1)))
        lines_to_be_checked.append(list(range(self.board_len-1, len(board)-1, self.board_len-1)))
        
        winner = None
        
        for line in lines_to_be_checked:
            if all([board[i]==Game.first_move for i in line]):
                winner = Game.first_move
            elif all([board[i]==Game.second_move for i in line]):
                winner = Game.second_move
            
        return winner
    
    def playable(self):
        """ If the game is still able to be continued 
        """
        return (self.winner is None) and any(self.possible_boards())
    
    def empty_squares(self):
        """ Returns index of all the empty squares
        """
        return [i for i in range(len(self.board)) if self.board[i] == Game.empty]
    
    def __repr__(self):
        s = """
{} | {} | {}
---------
{} | {} | {}
---------
{} | {} | {}
"""
        return s.format(*self.board)

Then let's make an agent that could be trained! 

In [33]:
import random
from collections import defaultdict

class Agent:
    def __init__(self, game_class, er=0.1, lr=0.5, player="X"):
        self.new_game = game_class # call self.new_game() to make a new game
        self.er = er # the rate at which to try out new, potentially bad options
        self.lr = lr # the rate at which the program learns from results
        self.Q = defaultdict(lambda: 0.0) # the Q "matrix", key is a board cofig
        self.player = player # the player the agent is playing for
    
    def select_move(self, game, learning=True):
        """ Select the best move for the current player of the game, 
            regardless of who the Agent is playing for. If learning, 
            then experiment with random moves once in awhile
        """
        def min_board(possible_boards):
            m = min(possible_boards, key=lambda s: self.Q[s])
            possibles = [p for p in possible_boards if self.Q[p] == self.Q[m]]
            return random.choice(possibles)
        
        def max_board(possible_boards):
            m = max(possible_boards, key=lambda s: self.Q[s])
            possibles = [p for p in possible_boards if self.Q[p] == self.Q[m]]
            return random.choice(possibles)
        
        possible_boards = game.possible_boards()
        if game.player == self.player:
            choose_board = max_board(possible_boards)
        else: # choose the worst possible play for the player, which is the best for the opponent
            choose_board = min_board(possible_boards)
        
        if learning:
            if random.random() < self.er: # if experiment
                choose_board = random.choice(possible_boards)

        return choose_board
    
    def learn_from_move(self, game, move):
        """ Update the Q matrix based on the result of the move
        """
        game.move(move)
        r = self.reward(game)
        
        next_state_value = 0.0
        if game.playable():
            next_state_value = self.Q[self.select_move(game, False)]
        self.Q[move] = (1-self.lr)*self.Q[move] + self.lr * (r+next_state_value)
    
    def learn(self, epochs=1000):
        """ Trains the agent 
        """
        for _ in range(epochs):
            g = self.new_game()
            while g.playable():
                move = self.select_move(g)
                self.learn_from_move(g, move)
    
    def reward(self, game):
        """ Rewards the action if it leads to a win 
        """
        if game.winner is None:
            return 0.0
        if game.winner == self.player:
            return 1.0
        else:
            return -1.0
    
    def play(self, show_text=True):
        """ Play a game where the AI does both sides 
        """
        g = self.new_game()
        turn = 0
        
        if show_text:
            print("Turn {}".format(turn))
            print(g)
            
        while g.playable():
            move = self.select_move(g, False)
            g.move(move)
            turn += 1
            
            if show_text:
                print("Turn {}".format(turn))
                print(g)
            
        if g.winner:
            if show_text:
                print("Winner is {}".format(g.winner))
            return g.winner
        else:
            if show_text:
                print("Draw!")
            return '-'
    
    def stats(self, total_games=10000):
        """ Empirically decide how well/biased the AI plays
        """
        results = [self.play(False) for i in range(total_games)]
        return {k: results.count(k)/(total_games/100) for k in ['X', 'O', '-']}
    
    def play_human(self):
        """ Play an interactive game with a human player 
        """
        g = self.new_game()
        turn = 0
        while g.playable():
            if g.player == self.player:
                move = self.select_move(g, False)
            else:
                move = self.get_human_move(g)
            
            g.move(move)
            turn += 1
            
            print("Turn {}".format(turn))
            print(g)
            
    def get_human_move(self, game):
        """ IO function to get a move from the human 
        """
        allowed_moves = game.empty_squares()
        print(allowed_moves)
        human_move = None
        while human_move is None:
            index = int(input("Choose where to put your {}".format(game.player)))
            if index in allowed_moves:
                human_move = game.board[:index] + game.player + game.board[index+1:]
        return human_move
        

Some code to train the agent and evaluate it. 

In [35]:
a = Agent(Game, er=0.1, lr=1.0, player="X")

milestones = [0,1000,2000,4000,8000,16000,32000]
for i, m in enumerate(milestones):
    to_be_trained = m-milestones[i-1] if i!=0 else m
    a.learn(to_be_trained)
    print("After {} epochs".format(m))
    print(a.stats(1000))

After 0 epochs
{'-': 11.0, 'X': 59.0, 'O': 30.0}
After 1000 epochs
{'-': 10.2, 'X': 58.9, 'O': 30.9}
After 2000 epochs
{'-': 16.1, 'X': 51.4, 'O': 32.5}
After 4000 epochs
{'-': 51.9, 'X': 30.3, 'O': 17.8}
After 8000 epochs
{'-': 93.7, 'X': 5.7, 'O': 0.6}
After 16000 epochs
{'-': 99.8, 'X': 0.2, 'O': 0.0}
After 32000 epochs
{'-': 99.9, 'X': 0.1, 'O': 0.0}


Fight me bro!

In [ ]:
a.play_human()